In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
from tensorflow.keras.utils import load_img, img_to_array

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
# Loading the dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "./train",
    image_size=(128, 128),
    batch_size=32
)

class_names = train_dataset.class_names
print("All model classes:", class_names)

Found 3115 files belonging to 36 classes.
All model classes: ['apple', 'banana', 'beetroot', 'bell pepper', 'cabbage', 'capsicum', 'carrot', 'cauliflower', 'chilli pepper', 'corn', 'cucumber', 'eggplant', 'garlic', 'ginger', 'grapes', 'jalepeno', 'kiwi', 'lemon', 'lettuce', 'mango', 'onion', 'orange', 'paprika', 'pear', 'peas', 'pineapple', 'pomegranate', 'potato', 'raddish', 'soy beans', 'spinach', 'sweetcorn', 'sweetpotato', 'tomato', 'turnip', 'watermelon']


In [4]:
# Arrays created for checking whether it's a fruit or veggie later on
fruits = ["apple", "banana", "grapes", "kiwi", "lemon", "mango", "orange", "pear", "pineapple", "pomegranate", "watermelon"]
vegetables = ["beetroot", "bell pepper", "cabbage", "capsicum", "carrot", "cauliflower", "chilli pepper", "corn", 
              "cucumber", "eggplant", "garlic", "ginger", "jalepeno", "lettuce", "onion", "paprika", "peas", 
              "potato", "raddish", "soy beans", "spinach", "sweetcorn", "sweetpotato", "tomato", "turnip"]



In [ ]:
# Create data augmentation layer
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
])

# Optimize data pipeline with caching and prefetching
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

# Create validation split
val_size = int(0.2 * 3115)  # 20% for validation
train_size = 3115 - val_size

train_ds = train_dataset.take(train_size)
val_ds = train_dataset.skip(train_size)

# Create the model with improvements
model = models.Sequential([
    layers.Rescaling(1./255, input_shape=(128, 128, 3)),
    data_augmentation,
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(len(class_names), activation='softmax')  # One neuron per class
])

# Model summary to check architecture
model.summary()

# Define callbacks WITHOUT TensorBoard
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath="fruit_veg_model_checkpoint",
        save_best_only=True,
        monitor="val_accuracy"
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=3,
        restore_best_weights=True
    )
]

# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model with validation data and callbacks
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    callbacks=callbacks
)


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 128, 128, 3)       0         
                                                                 
 sequential (Sequential)     (None, 128, 128, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 128, 128, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 64)       0         
 2D)                                                  

In [ ]:
# Evaluate on test dataset if available
try:
    test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        './test',
        image_size=(128, 128),
        batch_size=32
    )
    
    # Apply same preprocessing as training data
    test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)
    
    # Evaluate the model
    test_loss, test_accuracy = model.evaluate(test_dataset)
    print(f"Test accuracy: {test_accuracy:.2f}")
except:
    print("No test directory found or error loading test data")

Found 359 files belonging to 36 classes.
12/12 [==============================] - 3s 85ms/step - loss: 1.3420 - accuracy: 0.6017
Test accuracy: 0.60


In [ ]:
# Function to predict on new images
def predict_image(loaded_model, img_path):
    try:
        img = load_img(img_path, target_size=(128, 128))
        img_array = img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)

        predictions = loaded_model.predict(img_array)
        predicted_index = np.argmax(predictions[0])
        predicted_class = class_names[predicted_index]
        confidence = float(predictions[0][predicted_index])

        print(f"Model prediction: {predicted_class} (Confidence: {confidence:.2f})")
        
        # Check if the predicted class is a fruit or vegetable
        if predicted_class.lower() in fruits:
            return "Fruit", predicted_class, confidence
        elif predicted_class.lower() in vegetables:
            return "Vegetable", predicted_class, confidence
        else:
            return "Unknown category", predicted_class, confidence
        
    except Exception as e:
        print(f"Error in prediction: {e}")
        return None, None, None

In [ ]:
# Making sure the folder exists
save_path = r"C:\Users\abios\Desktop\frootVeggieClassifier.h5" # Note: pip install h5py
# Save the trained model
model.save(save_path)
print("Model saved successfully")

# Example usage
try:
    category, name, conf = predict_image(model, "./input/app.jpeg")
    print(f"Category: {category}, Name: {name}, Confidence: {conf:.2f}")
except Exception as e:
    print(f"Error with example prediction: {e}")

Model saved successfully
1/1 [==============================] - 0s 83ms/step
Model prediction: pomegranate (Confidence: 0.62)
Category: Fruit, Name: pomegranate, Confidence: 0.62


In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
model_path = r"C:\Users\abios\Desktop\frootVeggieClassifier.h5"
model = load_model(model_path)
print("Model loaded successfully")

# Example usage
try:
    category, name, conf = predict_image(model, "./input/let.jpg")
    print(f"Category: {category}, Name: {name}, Confidence: {conf:.2f}")
except Exception as e:
    print(f"Error with example prediction: {e}")